In [3]:
import pandas as pd
import json

In [44]:
# Datasets 
csv_dataset = pd.read_csv("./Data sets/amazon.csv")
excel_dataset = pd.read_excel('./Data sets/Sales.xlsx', sheet_name='Orders')
json_dataset = pd.read_json("./Data sets/steamgames.json")

In [45]:
print("CSV Dataset Columns:", csv_dataset.columns)
print("Excel Dataset Columns:", excel_dataset.columns)
print("JSON Dataset Columns:", json_dataset.columns)

CSV Dataset Columns: Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'img_link', 'product_link'],
      dtype='object')
Excel Dataset Columns: Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Delivery Duration',
       'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country',
       'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Discount Value', 'Profit', 'COGS'],
      dtype='object')
JSON Dataset Columns: Index(['name', 'discount', 'orig_price', 'disc_price', 'tags', 'reviews',
       'link'],
      dtype='object')


In [46]:
json_dataset.rename(columns={'name': 'product_name', 'orig_price': 'actual_price', 'disc_price': 'discounted_price'}, inplace=True)
excel_dataset.rename(columns={'Product Name': 'product_name', 'Sales': 'actual_price'}, inplace=True)

In [47]:
csv_df = csv_dataset[['product_name', 'actual_price']]
excel_df = excel_dataset[['product_name', 'actual_price']]
json_df = json_dataset[['product_name', 'actual_price', 'discounted_price']]

In [48]:
json_df['actual_price'] = json_df['actual_price'].fillna(json_df['discounted_price'])

C:\Users\moinu\AppData\Local\Temp\ipykernel_6716\361536175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['actual_price'] = json_df['actual_price'].fillna(json_df['discounted_price'])


In [49]:
json_df = json_df.drop(columns=['discounted_price'])

In [50]:
merged_df = pd.merge(csv_df, excel_df, on='product_name', how='outer')
merged_df = pd.merge(merged_df, json_df, on='product_name', how='outer')

In [51]:
merged_df['actual_price'] = merged_df[['actual_price_x', 'actual_price_y', 'actual_price']].bfill(axis=1).iloc[:, 0]

C:\Users\moinu\AppData\Local\Temp\ipykernel_6716\3366112267.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['actual_price'] = merged_df[['actual_price_x', 'actual_price_y', 'actual_price']].bfill(axis=1).iloc[:, 0]


In [52]:
merged_df = merged_df[['product_name', 'actual_price']]

In [53]:
merged_df.dropna(subset=['actual_price'], inplace=True)


In [54]:
merged_df

,product_name,actual_price
0,!!1000 Watt/2000-Watt Room Heater!! Fan Heater...,"₹1,599"
1,!!HANEUL!!1000 Watt/2000-Watt Room Heater!! Fa...,"₹1,599"
2,"""While you Were Out"" Message Book, One Form pe...",7.42
3,"""While you Were Out"" Message Book, One Form pe...",8.904
4,"""While you Were Out"" Message Book, One Form pe...",8.904
...,...,...
14182,骸心 Mainbody,99.0
14183,눈 떠보니 임진왜란이었다 - Back To the Joseon,239.0
14184,미연시 시뮬레이터 : 미소녀 게임의 주인공을 조종하는 게임,299.0
14185,피랍 일지 - 그 남자로부터의 탈출,329.0


In [55]:
merged_df.to_csv('combined_dataset.csv', index=False)